<a href="https://colab.research.google.com/github/muneeb-kk/Projects/blob/main/Langchain_RAG_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
%pip install --upgrade --quiet langchain-google-genai langchain-pinecone
%pip install langchain pinecone-client google-generativeai tqdm langchain_community pypdf PyPDF2 --quiet


In [4]:
import langchain
import pinecone
import google.generativeai
from PyPDF2 import PdfReader

In [5]:
from google.colab import userdata
from pinecone import Pinecone, ServerlessSpec
pinecone_api_key = userdata.get('PINECONE_API_KEY')
pc = Pinecone(api_key=pinecone_api_key)

In [6]:
index_name = "langchain-rag-project"

pc.delete_index(index_name)

pc.create_index(

    name=index_name,

    dimension=768,

    metric='cosine',

    spec  = ServerlessSpec(cloud="aws",region="us-east-1"),

)



index = pc.Index(index_name)

In [7]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings
import os
gen_ai_key = userdata.get('GOOGLE_API_KEY')

embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001", google_api_key= gen_ai_key)

In [33]:
from langchain.document_loaders import PyPDFLoader
import os
from langchain_community.document_loaders import PyPDFium2Loader

def load_pdf(file_path):
    try:
        if not os.path.exists(file_path):
            print(f"Error: File not found: {file_path}")
            return None
        print(f"Trying to load using PyPDFLoader")
        loader = PyPDFLoader(file_path)
        documents = loader.load()
        return documents
    except Exception as e:
        print(f"Error loading with PyPDFLoader: {e}")
        print("Trying with PyPDFium2Loader")
        try:
            loader = PyPDFium2Loader(file_path)
            documents = loader.load()
            return documents
        except Exception as e2:
            print(f"Error loading with PyPDFium2Loader: {e2}")
            return None

file_path = "/content/fitness_tracker.pdf"
documents = load_pdf(file_path)

if documents:
    print(f"Document loaded successfully. Number of pages: {len(documents)}")
else:
    print("Failed to load document.")

Trying to load using PyPDFLoader
Document loaded successfully. Number of pages: 38


In [34]:
len(documents)

38

In [43]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)
chunks = text_splitter.split_documents(documents)

In [44]:
from google.colab import userdata

pinecone_api_key = userdata.get('PINECONE_API_KEY')



In [45]:
from langchain_pinecone import PineconeVectorStore
vector_store = PineconeVectorStore(
    index=index,
    embedding=embeddings
)

In [46]:
retriever = vector_store.as_retriever(search_type="similarity", search_kwargs={"k": 5})

In [51]:
llm = ChatGoogleGenerativeAI(
    model="gemini-2.0-flash-exp",  # Specify the model here
    api_key=gen_ai_key,  # Assuming gen_ai_key holds your API key
    temperature=0.3,
    max_tokens=1000
)
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=retriever,
    return_source_documents=True,
)

In [52]:
import uuid
from uuid import uuid4
for i, chunk in enumerate(chunks):

    text_chunk = chunk.page_content if isinstance(chunk, langchain.schema.Document) else str(chunk)
    vector = embeddings.embed_documents([text_chunk])[0]
    index.upsert([(f"doc-{i}", vector, {"text": text_chunk})])


In [56]:
query = "Tell me about user 1?"

result = qa_chain({"query": query})

answer = result['result']
source_documents = result['source_documents']

print(answer)


User 1 has the following data: 5100, 1550, 7, Activity 1, Activity 2.

